# The Steepener Trade


In [28]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt


In [29]:
import warnings
warnings.filterwarnings("ignore")


In [30]:
xlsx_path = "steepener_trade_2024-01-02.xlsx"
LONG_FACE_2Y = 50_000_000  # face value long in 2Y
BOND_FACE = 100           # each bond is 100 face

In [31]:

info  = pd.read_excel(xlsx_path, "info")
dirty = pd.read_excel(xlsx_path, "dirty price")
clean = pd.read_excel(xlsx_path, "clean price")
dur   = pd.read_excel(xlsx_path, "duration")

id2  = int(info.loc[0, "KYTREASNO"])
id10 = int(info.loc[1, "KYTREASNO"])
cpn2 = float(info.loc[0, "cpn rate"])   # percent annual coupon
cpn10= float(info.loc[1, "cpn rate"])   # percent annual coupon



In [32]:
def set_dt_index(df):
    df["quote date"] = pd.to_datetime(df["quote date"])
    return df.set_index("quote date").sort_index()

dirty = set_dt_index(dirty)
clean = set_dt_index(clean)
dur   = set_dt_index(dur)

idx = dirty.index.intersection(dur.index).intersection(clean.index)

P2   = dirty.loc[idx, id2].astype(float)
P10  = dirty.loc[idx, id10].astype(float)
D2   = dur.loc[idx, id2].astype(float)
D10  = dur.loc[idx, id10].astype(float)
C2   = clean.loc[idx, id2].astype(float)
C10  = clean.loc[idx, id10].astype(float)



In [33]:
# ---------- 1.1/1.2: duration-neutral hedge each day ----------
hedge_ratio = (P2 * D2) / (P10 * D10)         # face10 / face2
face2 = pd.Series(LONG_FACE_2Y, index=idx)    # keep 50mm face constant
face10 = hedge_ratio * face2                  # short this much face of 10Y

num_bonds_2y = face2 / BOND_FACE
num_bonds_10y = face10 / BOND_FACE

# Market value (dirty) of short 10Y position 
short_mv_10y = (face10 / BOND_FACE) * P10




In [34]:
# ---------- 1.3: price PnL (position held at t-1) ----------
dP2  = P2.diff()
dP10 = P10.diff()

pnl2  = (face2.shift(1) / BOND_FACE) * dP2
pnl10 = -(face10.shift(1) / BOND_FACE) * dP10   # minus because short
pnl_net = pnl2 + pnl10



In [35]:
# ---------- 1.5: coupons ----------

coupon_per_bond_2y = cpn2 / 2.0
coupon_per_bond_10y = cpn10 / 2.0

# Dates where clean == dirty
tol = 1e-8
matches = ((C2 - P2).abs() < tol) & ((C10 - P10).abs() < tol)
coupon_pay_dates = matches[matches].index
coupon_dates = coupon_pay_dates - pd.tseries.offsets.BDay(1)

coupon2 = pd.Series(0.0, index=idx)
coupon10 = pd.Series(0.0, index=idx)

for dt in coupon_dates:
    if dt in idx:
        # coupon at dt depends on position at dt-1 
        coupon2.loc[dt]  = num_bonds_2y.shift(1).loc[dt]  * coupon_per_bond_2y
        coupon10.loc[dt] = -num_bonds_10y.shift(1).loc[dt] * coupon_per_bond_10y  # short pays coupon

coupon_net = coupon2 + coupon10
pnl_net_with_coupons = pnl_net + coupon_net



In [36]:

out = pd.DataFrame({
    "P2_dirty": P2, "P10_dirty": P10,
    "D2": D2, "D10": D10,
    "hedge_ratio_face10_over_face2": hedge_ratio,
    "face2_long": face2,
    "face10_short": face10,
    "num_bonds_2y": num_bonds_2y,
    "num_bonds_10y": num_bonds_10y,
    "pnl2_price": pnl2,
    "pnl10_price": pnl10,
    "pnl_net_price": pnl_net,
    "coupon2": coupon2,
    "coupon10": coupon10,
    "coupon_net": coupon_net,
    "pnl_net_with_coupons": pnl_net_with_coupons,
})




# ---------- Plot 1.2: hedge ratio ----------
plt.figure()
out["hedge_ratio_face10_over_face2"].dropna().plot()
plt.title("Hedge ratio (face10 / face2) over time")
plt.tight_layout()
plt.show()

# ---------- Plot 1.2: bond number ----------
plt.figure()
out["num_bonds_2y"].dropna().plot(label="2Y bonds (long)")
out["num_bonds_10y"].dropna().plot(label="10Y bonds (short magnitude)")
plt.legend()
plt.title("Bond counts over time (duration-neutral daily hedge)")
plt.tight_layout()
plt.show()

# ---------- Plot 1.4: daily PnL ----------
plt.figure()
out["pnl2_price"].dropna().plot(label="2Y PnL")
out["pnl10_price"].dropna().plot(label="10Y PnL")
out["pnl_net_price"].dropna().plot(label="Net PnL")
plt.legend()
plt.title("Daily PnL (price-only)")
plt.tight_layout()
plt.show()

# ---------- Plot 1.5: cumulative net PnL with coupons ----------
plt.figure()
out["pnl_net_with_coupons"].dropna().cumsum().plot()
plt.title("Cumulative net PnL (with coupons)")
plt.tight_layout()
plt.show()


first_dt = out.index.min()
last_dt  = out.index.max()

print("First date:", first_dt.date())
print("Last date:", last_dt.date())

print("\n1.1 initial hedge (first date):")
print("Hedge ratio:", out.loc[first_dt, "hedge_ratio_face10_over_face2"])
print("Short 10Y face:", out.loc[first_dt, "face10_short"])
print("Short 10Y MV (dirty):", (out.loc[first_dt, "face10_short"] / 100) * out.loc[first_dt, "P10_dirty"])
print("2Y bonds:", out.loc[first_dt, "num_bonds_2y"])
print("10Y bonds:", out.loc[first_dt, "num_bonds_10y"])

print("\n1.2 last date values:")
print("Hedge ratio:", out.loc[last_dt, "hedge_ratio_face10_over_face2"])
print("Short 10Y face:", out.loc[last_dt, "face10_short"])
print("2Y bonds:", out.loc[last_dt, "num_bonds_2y"])
print("10Y bonds:", out.loc[last_dt, "num_bonds_10y"])


pnl_first_day = out["pnl_net_price"].dropna().index.min()
pnl_last_day  = out["pnl_net_price"].dropna().index.max()

print("\n1.3 first PnL day:", pnl_first_day.date())
print("2Y:", out.loc[pnl_first_day, "pnl2_price"],
      "10Y:", out.loc[pnl_first_day, "pnl10_price"],
      "Net:", out.loc[pnl_first_day, "pnl_net_price"])

print("\n1.3 last PnL day:", pnl_last_day.date())
print("2Y:", out.loc[pnl_last_day, "pnl2_price"],
      "10Y:", out.loc[pnl_last_day, "pnl10_price"],
      "Net:", out.loc[pnl_last_day, "pnl_net_price"])

print("\nCumulative net (no coupons):", out["pnl_net_price"].dropna().sum())
print("Cumulative net (with coupons):", out["pnl_net_with_coupons"].dropna().sum())


First date: 2023-11-09
Last date: 2025-05-30

1.1 initial hedge (first date):
Hedge ratio: 0.24005990962159424
Short 10Y face: 12002995.481079713
Short 10Y MV (dirty): 11882965.526268914
2Y bonds: 500000.0
10Y bonds: 120029.95481079712

1.2 last date values:
Hedge ratio: 0.07938947624766349
Short 10Y face: 3969473.8123831744
2Y bonds: 500000.0
10Y bonds: 39694.73812383175

1.3 first PnL day: 2023-11-10
2Y: -19021.739130437254 10Y: 25318.818592902517 Net: 6297.079462465263

1.3 last PnL day: 2025-05-30
2Y: 15615.85580110858 10Y: -11498.135002501662 Net: 4117.720798606919

Cumulative net (no coupons): 3043663.4934246405
Cumulative net (with coupons): 3043663.4934246405


In [37]:
print(out.head())

             P2_dirty   P10_dirty        D2       D10  \
quote date                                              
2023-11-09  90.978261   99.000000  2.135193  8.173725   
2023-11-10  90.940217   98.789062  2.132450  8.168483   
2023-11-13  91.005774   98.882812  2.124233  8.160898   
2023-11-14  91.397418  100.414062  2.121523  8.174992   
2023-11-15  91.253906   99.742188  2.118771  8.164722   

            hedge_ratio_face10_over_face2  face2_long  face10_short  \
quote date                                                            
2023-11-09                       0.240060    50000000  1.200300e+07   
2023-11-10                       0.240317    50000000  1.201585e+07   
2023-11-13                       0.239559    50000000  1.197794e+07   
2023-11-14                       0.236211    50000000  1.181054e+07   
2023-11-15                       0.237419    50000000  1.187094e+07   

            num_bonds_2y  num_bonds_10y     pnl2_price    pnl10_price  \
quote date                   

In [38]:
print(out.tail())



             P2_dirty   P10_dirty        D2       D10  \
quote date                                              
2025-05-23  97.904452  100.762021  0.607258  7.147798   
2025-05-27  97.907937  101.224864  0.596299  7.140169   
2025-05-28  97.913829  100.967561  0.593559  7.135270   
2025-05-29  97.917768  101.397758  0.590819  7.135881   
2025-05-30  97.949000  101.685377  0.588080  7.135339   

            hedge_ratio_face10_over_face2  face2_long  face10_short  \
quote date                                                            
2025-05-23                       0.082548    50000000  4.127399e+06   
2025-05-27                       0.080777    50000000  4.038834e+06   
2025-05-28                       0.080671    50000000  4.033532e+06   
2025-05-29                       0.079954    50000000  3.997699e+06   
2025-05-30                       0.079389    50000000  3.969474e+06   

            num_bonds_2y  num_bonds_10y    pnl2_price   pnl10_price  \
quote date                     

In [ ]:

import plotly.graph_objects as go

def line_plot(df, cols, title, yaxis_title=""):
    fig = go.Figure()
    for c in cols:
        s = df[c].dropna()
        fig.add_trace(go.Scatter(x=s.index, y=s.values, mode="lines", name=c))
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title=yaxis_title,
        template="plotly_white",
        height=420
    )
    fig.show()

# 1.2 hedge ratio
line_plot(out, ["hedge_ratio_face10_over_face2"], "Hedge ratio (face10 / face2) over time")

# 1.2 bond counts
line_plot(out, ["num_bonds_2y", "num_bonds_10y"], "Bond counts over time (duration-neutral daily hedge)")

# 1.4 daily PnL (price-only)
line_plot(out, ["pnl2_price", "pnl10_price", "pnl_net_price"], "Daily PnL (price-only)")

# 1.5 cumulative net PnL with coupons
cum = out["pnl_net_with_coupons"].dropna().cumsum()
fig = go.Figure()
fig.add_trace(go.Scatter(x=cum.index, y=cum.values, mode="lines", name="Cumulative net PnL (with coupons)"))
fig.update_layout(title="Cumulative net PnL (with coupons)", xaxis_title="Date", template="plotly_white", height=420)
fig.show()

#Cumulative net PnL without coupons
s = out["pnl_net_price"].dropna().cumsum()
fig = go.Figure(go.Scatter(x=s.index, y=s.values, mode="lines", name="Cumulative PnL (no coupons)"))
fig.update_layout(title="Cumulative net PnL (price-only, no coupons)", xaxis_title="Date", template="plotly_white")
fig.show()
